# Test on a different robot

In [ ]:
from pathlib import Path
import time

import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import model_from_json

# General settings
sns.set_style('whitegrid')

In [ ]:
data_dir = '../../data'
data_name = 'SonyAIBORobotSurface2'
data_filename = data_dir+'/'+data_name+'/'+'SonyAIBORobotSurface2_TEST.txt'
robot_data = np.loadtxt(Path(data_filename))
print('Loaded from', data_filename)

y = robot_data[:,0]
x = robot_data[:,1:]
print('The shape of x is', x.shape)
print('The shape of y is', y.shape)

# Change from classes 1 and 2 to classes 0 and 1
y = (y - y.min())/(y.max()-y.min())

print('Number of samples of class 0', (y == 0).sum())
print('Number of samples of class 1', (y == 1).sum())
n0 = (y == 0).sum()
n1 = (y == 1).sum()
print('Baseline accuracy to beat:', n1/(n0+n1))

In [ ]:
x_reshape = np.zeros((x.shape[0], 70)) 
x_reshape[:x.shape[0],:x.shape[1]] = x
x_test = x_reshape
y_test = y

In [ ]:
samples = [1, 2, 3, 5, 7]
for i in samples:
    plt.plot(x_test[i], label='class'+str(y_test[i]))
plt.legend(loc='upper right', frameon=False)
plt.ylim([-3.5, 3.5])

## Load model

In [ ]:
json_file = open('model.json', 'r')
loaded_json_model = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_json_model)
# load weights into new model
loaded_model.load_weights('model.h5')
print('Model loaded from file')
optimizer = keras.optimizers.Adam()
loaded_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print('Model compiled')

In [ ]:
result = loaded_model.evaluate(x_test, y_test, batch_size=batch_size)
print('Accuracy on robot 2 data is', result[1])
y_probability = loaded_model.predict_on_batch(x_test)
y_predicted_class = np.round(y_probability).flatten()
print('Some of the test results:')
print('True', y_test[:23])
print('Pred', y_predicted_class[:23])